Imports

In [2]:
from __future__ import print_function
import os
import numpy as np
import sys
import subprocess
import tarfile
import xml.etree.ElementTree as ET
from IPython.display import display, Image
from scipy import ndimage
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
from six.moves import range
from yaafelib import *

In [21]:
class IsmisFeatures:
    
    def __init__(self, audiofile):
        self.audiofile = audiofile
        
        self.temporal_centroid = None      #1: Temporal Centroid
        self.spectral_centroid = None      #2: Spectral Centroid average value
        self.ase_per_band_avg = []         #4-37: Audio Spectrum Envelope (ASE) average values in 34 frequency bands
        self.ase_avg = None                #38: ASE average value (averaged for all frequency bands)
        self.ase_per_band_var = []         #39-72: ASE variance values in 34 frequency bands
        self.ase_var_avg = None            #73: averaged ASE variance parameters
        self.centroid_avg = None           #74: Audio Spectrum Centroid – average
        self.centroid_var = None           #75: Audio Spectrum Centroid – variance
        self.spread_avg = None             #76: Audio Spectrum Spread – average
        self.spread_var = None             #77: Audio Spectrum Spread – variance
        self.sfm_per_band_avg = []         #78-101: Spectral Flatness Measure (SFM) average values for 24 frequency bands
        self.sfm_avg = None                #102: SFM average value (averaged for all frequency bands)
        self.sfm_per_band_var = []         #103-126: Spectral Flatness Measure (SFM) variance values for 24 frequency bands
        self.sfm_var_avg = None            #127: averaged SFM variance parameters
        self.mfcc = []                     #128-147: 20 first mel cepstral coefficients average values
        self.fv = []
        
    def export_features(self):
        self.fv.append(self.temporal_centroid)
        self.fv.append(self.spectral_centroid)
        self.fv.extend(filter(lambda a: a != 0, self.ase_per_band_avg))
        self.fv.append(self.ase_avg)
        self.fv.extend(filter(lambda a: a != 0, self.ase_per_band_var))
        self.fv.append(self.ase_var_avg)
        self.fv.append(self.centroid_avg)
        self.fv.append(self.centroid_var)
        self.fv.append(self.spread_avg)
        self.fv.append(self.spread_var)
        self.fv.extend(self.sfm_per_band_avg)
        self.fv.append(self.sfm_avg)
        self.fv.extend(self.sfm_per_band_var)
        self.fv.append(self.sfm_var_avg)
        self.fv.extend(self.mfcc.tolist()[0])
    
        
    def extract_features(self):
        if os.path.exists(self.audiofile):
            print('Getting features from ' + self.audiofile)
        else:
            raise Exception('File ' + self.audiofile + ' not found')
            
        self.extract_mpeg7_features()
        self.extract_mfcc()
        
    def extract_mpeg7_features(self):
        ns = {'xmlns' : 'urn:mpeg:mpeg7:schema:2001',
              'mpeg7' : 'urn:mpeg:mpeg7:schema:2001',
              'xsi' : 'http://www.w3.org/2001/XMLSchema-instance',
              'xsi:schemaLocation' : 'urn:mpeg:mpeg7:schema:2001 http://mpeg7audioenc.sourceforge.net/mpeg7audioenc.xsd'} 

        result = subprocess.check_output(['java', '-jar', 'MPEG7AudioEnc.jar', self.audiofile, 'mpeg7config.xml'])
        root = ET.fromstring(result)

        temporal_centroid_xml = root.find(".//mpeg7:AudioDescriptor[@xsi:type='TemporalCentroidType']", ns)
        spectral_cetroid_xml = root.find(".//mpeg7:AudioDescriptor[@xsi:type='SpectralCentroidType']", ns)
        audio_spectrum_centroid = root.find(".//mpeg7:AudioDescriptor[@xsi:type='AudioSpectrumCentroidType']", ns)
        audio_spectrum_spread = root.find(".//mpeg7:AudioDescriptor[@xsi:type='AudioSpectrumSpreadType']", ns)
        audio_spectrum_envelope = root.find(".//mpeg7:AudioDescriptor[@xsi:type='AudioSpectrumEnvelopeType']", ns)
        audio_spectrum_flatness = root.find(".//mpeg7:AudioDescriptor[@xsi:type='AudioSpectrumFlatnessType']", ns)

        self.temporal_centroid = self.parse_xml_scalar(temporal_centroid_xml, ns)
        self.spectral_centroid = self.parse_xml_scalar(spectral_cetroid_xml, ns)
        
        envelope_values = self.parse_2d_xml_vector(audio_spectrum_envelope, ns)          
        self.ase_per_band_avg = [np.mean(band) for band in envelope_values]
        self.ase_avg = np.mean(self.ase_per_band_avg)
        self.ase_per_band_var = [np.var(band) for band in envelope_values]
        self.ase_var_avg = np.mean(self.ase_per_band_var)
        
        centroid_values = self.parse_xml_vector(audio_spectrum_centroid, ns)
        self.centroid_avg = np.mean(centroid_values)
        self.centroid_var = np.var(centroid_values)
        
        spread_values = self.parse_xml_vector(audio_spectrum_spread, ns)
        self.spread_avg = np.mean(spread_values)
        self.spread_var = np.var(spread_values)
        
        flatness_values = self.parse_2d_xml_vector(audio_spectrum_flatness, ns)
        self.sfm_per_band_avg = [np.mean(band) for band in flatness_values]
        self.sfm_avg = np.mean(self.sfm_per_band_avg)
        self.sfm_per_band_var = [np.var(band) for band in flatness_values]
        self.sfm_var_avg = np.mean(self.ase_per_band_var)
    
    def parse_xml_scalar(self, element, ns):
        return float(element.find(".//mpeg7:Scalar", ns).text)
    
    def parse_xml_vector(self, element, ns):
        values_splitted = element.find(".//mpeg7:Raw", ns).text.split()
        return map((lambda x: float(x)), values_splitted)
    
    def parse_2d_xml_vector(self, element, ns):
        values_string = element.find(".//mpeg7:Raw", ns).text
        values_splitted = [s.strip().split() for s in values_string.splitlines()]
        values = [map((lambda x: float(x)), value) for value in values_splitted] #cast to float
        transposed_values = list(map(list, zip(*values))) #transpose matrix to have 1 long vector per 1 band
        return transposed_values 
    
    def extract_mfcc(self):
        fp = FeaturePlan(sample_rate=22050, normalize=1)
        fp.addFeature('mfcc: MFCC CepsNbCoeffs=20')
        df = fp.getDataFlow()
        engine = Engine()
        engine.load(df)
        afp = AudioFileProcessor()
        afp.setOutputFormat('csv', 'features', {'Precision': '8', 'Metadata': 'False'})
        afp.processFile(engine, self.audiofile)
        engine.flush()
        feats = engine.readAllOutputs()
        self.mfcc = feats['mfcc']

In [22]:
ismisFeature = IsmisFeatures('genres/jazz/jazz.00012.au')
ismisFeature.extract_features()
ismisFeature.export_features()
print(ismisFeature.fv)

Getting features from genres/jazz/jazz.00012.au
[14.879065, 603.90454, 6.1979938182624088e-05, 4.3238988269679432e-05, 8.7111149326060414e-05, 9.7047038008353444e-05, 8.2408865526196952e-05, 4.1862386730993678e-05, 4.7521544575655228e-05, 9.3486067822837711e-05, 0.0001499305740994294, 9.9638428054004511e-05, 6.4180795873407825e-05, 8.0260679992284019e-05, 0.00012449686504224368, 0.00013930560039371437, 7.7534303003782635e-05, 5.1194918612017628e-05, 2.9396026003485197e-05, 5.3364923009808558e-05, 5.2094454918579105e-05, 3.081040830283429e-05, 5.3026028137770295e-05, 7.4569471586648472e-05, 1.9753754629466478e-05, 8.145540216180731e-06, 7.2729975399833266e-06, 4.1691593510086764e-06, 2.1374105675571837e-06, 1.7999177721573836e-06, 1.6187460069039695e-06, 6.0398149815671821e-07, 4.7706095929243068e-09, 4.9410756872453488e-05, 7.2731841866433517e-09, 3.7128288073298401e-09, 2.0853630631784872e-08, 3.0214601828047258e-08, 2.582525864284118e-08, 4.265275602738052e-09, 4.4436554345603977e-09

In [23]:
root = os.path.join(os.getcwd(),'genres')

genres = os.listdir(root)
mappings = dict(enumerate(genres))
mappings_rev = {v: k for k, v in mappings.items()}
dataset_xs = []
dataset_y = []

for folder in os.listdir(root):
    data_folder = os.path.join(root, folder)
    filenames = os.listdir(data_folder)
    for filename in filenames:
        path=os.path.join(data_folder, filename)
        genre = mappings_rev[folder]
        ismisFeature = IsmisFeatures(path)
        ismisFeature.extract_features()
        ismisFeature.export_features()
        dataset_xs.append(ismisFeature.fv)
        dataset_y.append(genre)

Getting features from /notebooks/genres/classical/classical.00015.au
Getting features from /notebooks/genres/classical/classical.00061.au
Getting features from /notebooks/genres/classical/classical.00012.au
Getting features from /notebooks/genres/classical/classical.00050.au
Getting features from /notebooks/genres/classical/classical.00098.au
Getting features from /notebooks/genres/classical/classical.00086.au
Getting features from /notebooks/genres/classical/classical.00002.au
Getting features from /notebooks/genres/classical/classical.00099.au
Getting features from /notebooks/genres/classical/classical.00054.au
Getting features from /notebooks/genres/classical/classical.00090.au
Getting features from /notebooks/genres/classical/classical.00060.au
Getting features from /notebooks/genres/classical/classical.00091.au
Getting features from /notebooks/genres/classical/classical.00006.au
Getting features from /notebooks/genres/classical/classical.00077.au
Getting features from /notebooks/g

In [24]:
print(dataset_xs[0])

[17.088455, 700.4137, 1.7853434456206413e-05, 1.0813802362571563e-06, 1.7851488395210774e-06, 2.1456009317862401e-06, 1.9372303071908537e-06, 6.2867647930485767e-07, 6.3576729051859258e-07, 3.0782149292569537e-06, 3.992124629599439e-06, 1.9469113129586451e-06, 4.0860488911565291e-06, 9.2697636219726249e-06, 1.867760219088634e-05, 3.5671773831141524e-05, 3.4716686946492038e-05, 6.9041636202141423e-06, 2.328742180204465e-05, 2.9595260806659184e-05, 1.2423437670124788e-05, 2.9309029997591238e-06, 1.6652349842660617e-06, 2.0928381968261656e-06, 9.6890629581357855e-07, 8.3025144163671821e-07, 4.2231480037641839e-07, 1.3355725831954672e-07, 5.2736771393888003e-08, 4.3190853202497449e-08, 2.345522852732912e-08, 4.0038775906888949e-09, 3.9694994025058341e-10, 6.4377776014983628e-06, 3.1292879253674955e-09, 4.510357233636496e-12, 1.2697571683096011e-11, 1.7300430501533603e-11, 1.3010072178737578e-11, 1.2792251220572329e-12, 2.1983469042409744e-12, 4.1697224615416006e-11, 1.1333071084507977e-10,

In [27]:
def evens(dataset):
    return dataset[::2]
def odds(dataset):
    return dataset[1::2]

train_dataset_xs = evens(dataset_xs)
train_dataset_y = evens(dataset_y)
test_dataset_xs = odds(dataset_xs)
test_dataset_y = odds(dataset_y)

def randomize(dataset_xs, dataset_y):
    permutation = np.random.permutation(len(dataset_y))
    shuffled_dataset_xs = np.asarray(dataset_xs)[permutation]
    shuffled_dataset_y = np.asarray(dataset_y)[permutation]
    return shuffled_dataset_xs, shuffled_dataset_y

train_dataset_xs, train_dataset_y = randomize(train_dataset_xs,train_dataset_y)
test_dataset_xs, test_dataset_y = randomize(test_dataset_xs, test_dataset_y)

In [28]:
try:
    f = open('data/gztan.pickle', 'wb')
    save = {
        'train_dataset_xs': train_dataset_xs,
        'train_dataset_y': train_dataset_y,
        'test_dataset_xs': test_dataset_xs,
        'test_dataset_y': test_dataset_y,
        'mappings': mappings
    }
    pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
    f.close()
except Exception as e:
    print('Unable to save data to', pickle_file, ':', e)
    raise